In [1]:
# main libraries
import os
import re
import sys
import glob
import shutil
from ixpeobssim.core import pipeline as ixpe
from ixpeobssim.binning.polarization import xBinnedPolarizationCube
from astropy.io import fits
import matplotlib.pyplot as plt

# warnings and text customization
import io
import warnings
from contextlib import redirect_stdout
from ixpeobssim.utils.logging_ import logger
from astropy.wcs.wcs import FITSFixedWarning
from astropy.io.fits.verify import VerifyWarning

logger.setLevel('WARNING')
warnings.filterwarnings('ignore', category=FITSFixedWarning)
warnings.filterwarnings('ignore', category=VerifyWarning)

>>> PyXSPEC is not installed, you will no be able to use it.
>>> No module named 'mido'
>>> mido is a library to manipulate MIDI files.
>>> See https://mido.readthedocs.io/en/latest/ for more details.
>>> Type `pip install --user mido` to install it


In [2]:
def get_time_range(fits_path):
    """Reads a FITS file and returns the min TSTART and max TSTOP from its GTI extension."""
    with fits.open(fits_path) as hdul:
        if 'GTI' not in hdul:
            return None, None
        gti_data = hdul['GTI'].data
        if gti_data is None or len(gti_data) == 0:
            return None, None
        t_start = gti_data['START'].min()
        t_stop = gti_data['STOP'].max()
        return t_start, t_stop

def get_detector(filename):
    """Extracts the detector string (e.g., 'det1') from a filename."""
    match = re.search(r'(det\d)', os.path.basename(filename))
    return match.group(1) if match else None

In [3]:
observation = '80_sim_evt'
obs_dir = f'/Users/leodrake/Documents/MIT/IXPE/{observation}'
os.chdir(obs_dir)
os.makedirs('reg', exist_ok=True)

bkg_handling_path = '/Users/leodrake/Documents/MIT/IXPE/IXPE-background/'

l2_events = sorted(glob.glob('event_l2/*[0-9][0-9].fits'))
region_file = f'reg/{observation}_src.reg'

#background rejection
BKG_REJECT = False

if BKG_REJECT:
    output_dir = os.path.join(obs_dir, 'event_l2_bkgrej')
    run_computation = True

    # check if the output directory exists and already contains rejected files
    if os.path.isdir(output_dir) and glob.glob(f'{output_dir}/*_rej.fits'):
        run_computation = False

    if run_computation:
        script_path = os.path.join(bkg_handling_path, 'reject_background.py')
        l1_events = sorted(glob.glob('event_l1/*[0-9][0-9].fits'))
    
        # mapping corresponding l1 files to l2 cycle based on overlapping GTIs (if necessary)
        l2_to_l1_mapping = {}
        for l2_event in l2_events:
            l2_start, l2_stop = get_time_range(l2_event)
            l2_detector = get_detector(l2_event)
            if l2_start is None or l2_detector is None: continue
    
            corresponding_l1s = []
            for l1_event in l1_events:
                l1_start, l1_stop = get_time_range(l1_event)
                l1_detector = get_detector(l1_event)
                if l1_start is None or l1_detector is None: continue
                
                if (l1_detector == l2_detector) and (l1_start < l2_stop and l2_start < l1_stop):
                    corresponding_l1s.append(l1_event)
            
            l2_to_l1_mapping[l2_event] = corresponding_l1s
        
        # mapping verification
        print("\n    L2——>L1 File Mapping Verification ")
        print("——————————————————————————————————————————")
        for l2_file, l1_files in l2_to_l1_mapping.items():
            l1_basenames = [os.path.basename(f) for f in l1_files]
            print(f"  L2 File: {os.path.basename(l2_file)} ——> Maps to L1 Files: {l1_basenames}")
        print("——————————————————————————————————————————")
        
        #runs background_filtering.py for each l2 event file
        for l2_event, l1_file_list in l2_to_l1_mapping.items():
            if not l1_file_list:
                print(f" Skipping {os.path.basename(l2_event)}, no corresponding L1 files found.")
                continue
                
            abs_l1_paths = [os.path.join(obs_dir, f) for f in l1_file_list]
            l1_files_str = ' '.join(abs_l1_paths)
    
            with redirect_stdout(io.StringIO()):
                !python {script_path} {os.path.join(obs_dir, l2_event)} {l1_files_str}
            
            
        # moves _rej files to separate dir
        os.makedirs(output_dir, exist_ok=True)
        source_dir = os.path.join(obs_dir, 'event_l2')
        bkgrej_files = glob.glob(f'{source_dir}/*_rej.fits')
    
        if not bkgrej_files:
            print("Warning: No background-rejected files ('*_rej.fits') were found to move.")
        else:
            for src_path in bkgrej_files:
                dest_path = os.path.join(output_dir, os.path.basename(src_path))
                shutil.move(src_path, dest_path)
    
    l2_events = sorted(glob.glob(f'{output_dir}/*_rej.fits'))

In [4]:
# xpselect args
emin_keV = 1.0
emax_keV = 15.0

# xpbin args
ebin_algorithm = 'PCUBE'
irf_name = 'ixpe:obssim20240701_alpha075:v13' #Current IRF as of June 10 2025
ebin_edges = [2,8]

with redirect_stdout(io.StringIO()):
    xpselect_output = ixpe.xpselect(
        *l2_events,
        emin=emin_keV,
        emax=emax_keV,
        regfile=region_file,
        ltimeupdate=True
    )
    
    xpbin_output = ixpe.xpbin(
        *xpselect_output, 
        suffix='-'.join(map(str, ebin_edges)) + 'kev',
        algorithm=ebin_algorithm,
        irfname=irf_name,
        acceptcorr=True,
        weights=True,
        ebinalg='LIST',
        ebins=1,
        ebinning=ebin_edges
    )

    pcube = xBinnedPolarizationCube(xpbin_output[0])
    
    for file in xpbin_output[1:]:
        next_cube = xBinnedPolarizationCube(file)
        pcube += next_cube

results_table = pcube.as_table()
full_table_string = '\n'.join(results_table.pformat(max_width=-1))
print(full_table_string)

with redirect_stdout(io.StringIO()):
    pcube.plot()

usage: ipykernel_launcher.py [-h] [--suffix SUFFIX] [--tmin TMIN]
                             [--tmax TMAX] [--tinvert {True,False}]
                             [--phasemin PHASEMIN] [--phasemax PHASEMAX]
                             [--phaseinvert {True,False}] [--emin EMIN]
                             [--emax EMAX] [--einvert {True,False}] [--ra RA]
                             [--dec DEC] [--rad RAD] [--innerrad INNERRAD]
                             [--regfile REGFILE] [--reginvert {True,False}]
                             [--mask MASK] [--mcsrcid MCSRCID]
                             [--mc {True,False}] [--ltimeupdate {True,False}]
                             [--ltimealg {LTSUM,LTSCALE}]
                             [--overwrite {True,False}]
                             filelist [filelist ...]
ipykernel_launcher.py: error: the following arguments are required: filelist


SystemExit: 2

/Users/leodrake/miniforge3/envs/heasoft-6.35.1/lib/python3.12/site-packages/IPython/core/interactiveshell.py:3678: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


In [5]:
# Helper function to get detector ID from a filename
def get_detector(filename):
    match = re.search(r'(det\d)', os.path.basename(filename))
    return match.group(1) if match else None

# --- 1. Define Paths and Initial Input Files ---
l2_events = sorted(glob.glob('event_l2_bkgrej/*_rej.fits'))
source_region_file = os.path.join(obs_dir, 'reg', f'{observation}_src.reg')
background_region_file = os.path.join(obs_dir, 'reg', f'{observation}_bkg.reg')
analysis_folder = os.path.join(obs_dir, 'bkg_subtraction')
bkg_handling_path = '/Users/leodrake/Documents/MIT/IXPE/IXPE-background'

# --- 2. Create the Analysis Directory ---
print(f"--- Preparing analysis directory at: {analysis_folder} ---")
os.makedirs(analysis_folder, exist_ok=True)


# --- 3. Create SOURCE Event Files ---
print("\n--- Creating source event files ---")
source_select_files = ixpe.xpselect(*l2_events, regfile=source_region_file, suffix='select', overwrite=True)

# --- CORRECTED: Move and rename to the required du{id}_src.fits format ---
print("\n--- Moving and renaming source files ---")
for file_path in source_select_files:
    detector_str = get_detector(file_path) # e.g., 'det1'
    if not detector_str: continue
    
    # Construct the simple name the script expects (e.g., 'du1_src.fits')
    new_name = f"du{detector_str[-1]}_src.fits"
    dest_path = os.path.join(analysis_folder, new_name)
    
    print(f"  Moving {os.path.basename(file_path)} to {dest_path}")
    shutil.move(file_path, dest_path)


# --- 4. Create BACKGROUND Event Files ---
print("\n--- Creating background event files ---")
bkg_select_files = ixpe.xpselect(*l2_events, regfile=background_region_file, suffix='bkg', overwrite=True)

# --- CORRECTED: Move and rename to the required du{id}_bkg.fits format ---
print("\n--- Moving background files ---")
for file_path in bkg_select_files:
    detector_str = get_detector(file_path) # e.g., 'det1'
    if not detector_str: continue

    # Construct the simple name the script expects (e.g., 'du1_bkg.fits')
    new_name = f"du{detector_str[-1]}_bkg.fits"
    dest_path = os.path.join(analysis_folder, new_name)

    print(f"  Moving {os.path.basename(file_path)} to {dest_path}")
    shutil.move(file_path, dest_path)
    

# --- 5. Verify the Prepared Directory ---
print("\n--- Verification: Contents of analysis directory ---")
!ls -l {analysis_folder}
print("--------------------------------------------------")
print("Preparation complete. Ready to run the subtraction script.")


# --- 6. Run the Background Subtraction Script ---
# This part remains the same, as it will now find the correctly named files.
print("\n--- Running Background Subtraction Analysis ---")
script_path = os.path.join(bkg_handling_path, 'subtract_background.py')
irf = 'ixpe:obssim20240701_alpha075:v13'
src_suffix = 'src'
bkg_suffix = 'bkg'
source_radius_arcsec = 80.0
background_inner_arcsec = 150.0
background_outer_arcsec = 300.0
energy_bins = "2.0 8.0"

# Construct the command using the prepared analysis_folder
command_to_run = f"""
{sys.executable} {script_path} \\
    --bkg_subtraction \\
    --irfname {irf} \\
    --filename {src_suffix} \\
    --bkg_filename {bkg_suffix} \\
    --folder_path {analysis_folder} \\
    --energy_binning {energy_bins} \\
    --src_radius {source_radius_arcsec} \\
    --bkg_inner_radius {background_inner_arcsec} \\
    --bkg_outer_radius {background_outer_arcsec}
"""

#with redirect_stdout(io.StringIO()):
#    !{command_to_run}

{command_to_run}

--- Preparing analysis directory at: /Users/leodrake/Documents/MIT/IXPE/04001001/bkg_subtraction ---

--- Creating source event files ---

    Welcome to ixpeobssim 31.0.3 (built on Thu, 03 Oct 2024 12:17:38 +0200).

    Copyright (C) 2015--2023, the ixpeobssim team.

    ixpeobssim comes with ABSOLUTELY NO WARRANTY.
    This is free software, and you are welcome to redistribute it under certain
    conditions. See the LICENSE file for details.

    Visit https://bitbucket.org/ixpesw/ixpeobssim for more information.

Filename: event_l2_bkgrej/ixpe04001001_det1_evt2_v01_rej.fits
No.    Name      Ver    Type      Cards   Dimensions   Format
  0  PRIMARY       1 PrimaryHDU      62   ()      
  1  EVENTS        1 BinTableHDU    270   372555R x 10C   [J, D, 16X, 16X, J, E, E, E, D, D]   
  2  GTI           1 BinTableHDU     53   730R x 2C   [D, D]   
Filename: event_l2_bkgrej/ixpe04001001_det1_evt2_v01_rej.fits
No.    Name      Ver    Type      Cards   Dimensions   Format
  0  PRIMARY      

{'\n/Users/leodrake/miniforge3/envs/heasoft-6.35.1/bin/python /Users/leodrake/Documents/MIT/IXPE/IXPE-background/subtract_background.py \\\n    --bkg_subtraction \\\n    --irfname ixpe:obssim20240701_alpha075:v13 \\\n    --filename src \\\n    --bkg_filename bkg \\\n    --folder_path /Users/leodrake/Documents/MIT/IXPE/04001001/bkg_subtraction \\\n    --energy_binning 2.0 8.0 \\\n    --src_radius 80.0 \\\n    --bkg_inner_radius 150.0 \\\n    --bkg_outer_radius 300.0\n'}